# 1-1. 데이터 로드 및 기본 정보 확인
신용카드 거래 데이터를 불러오고, 데이터의 구조 및 클래스 분포를 확인합니다.

In [ ]:
import pandas as pd

# 데이터 로드
df = pd.read_csv('creditcard.csv')

# 데이터 구조 확인
print(df.shape)
print(df.info())
print(df.head())

# 클래스 분포 확인
print(df['Class'].value_counts())

# 1-2. 클래스 분포 시각화
클래스 불균형을 시각적으로 확인합니다.

In [ ]:
import matplotlib.pyplot as plt

counts = df['Class'].value_counts()
labels = ['Normal', 'Fraud']

plt.figure(figsize=(6, 4))
plt.bar(labels, counts)
plt.title('Class Distribution')
plt.show()
plt.close('all')  # 리소스 정리

# 1-3. 정규화
거래 금액과 시간 데이터를 정규화하여 모델 학습에 적합한 형태로 변환합니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df['Amount_Scaled'] = scaler.fit_transform(df[['Amount']])
df['Time_Scaled'] = scaler.fit_transform(df[['Time']])

# 기존 피처 삭제
df = df.drop(['Amount', 'Time'], axis=1)

# 2-1. 데이터 분리
학습 데이터와 테스트 데이터를 나눕니다.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 2-2. 로지스틱 회귀 기본 학습
불균형 데이터를 그대로 학습하여 기본 성능을 확인합니다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# 3-1. SMOTE 적용 (Oversampling)
데이터 불균형 문제를 해결하기 위해 소수 클래스 데이터를 합성하여 샘플을 균등하게 만듭니다.

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(pd.Series(y_train_smote).value_counts())

# 3-2. SMOTE 재학습
SMOTE 적용 후 로지스틱 회귀 재학습을 통해 모델 성능을 비교합니다.

In [ ]:
model_smote = LogisticRegression(max_iter=1000)
model_smote.fit(X_train_smote, y_train_smote)

y_pred_smote = model_smote.predict(X_test)

print(confusion_matrix(y_test, y_pred_smote))
print(classification_report(y_test, y_pred_smote))

# 4-1. Isolation Forest 비지도 학습
비지도 학습으로 이상치(사기 거래)를 탐지합니다.

In [ ]:
from sklearn.ensemble import IsolationForest

iso_model = IsolationForest(contamination=0.001, random_state=42)
iso_model.fit(X_train)

# predict: 정상 1, 이상치 -1
y_pred_iso = iso_model.predict(X_test)
y_pred_iso = [1 if x == -1 else 0 for x in y_pred_iso]  # 이상치만 1로 변환

print(confusion_matrix(y_test, y_pred_iso))
print(classification_report(y_test, y_pred_iso))

# 5-1. AUC 비교
지도학습과 비지도학습의 AUC를 비교합니다.

In [ ]:
from sklearn.metrics import roc_auc_score

print('지도학습 AUC:', roc_auc_score(y_test, model_smote.predict_proba(X_test)[:, 1]))
print('비지도학습 AUC:', roc_auc_score(y_test, y_pred_iso))

# 6-1. 모델 튜닝 (Grid Search)
Grid Search를 통해 로지스틱 회귀의 최적 파라미터를 찾습니다.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=3, scoring='f1', n_jobs=-1)
grid.fit(X_train_smote, y_train_smote)

print('Best Parameters:', grid.best_params_)
print('Best Score:', grid.best_score_)

# 7-1. Stratified K-Fold Cross Validation
불균형 데이터를 고려한 K-Fold 교차 검증을 수행합니다.

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = LogisticRegression(max_iter=1000)
scores = cross_val_score(model, X, y, cv=kfold, scoring='f1', n_jobs=-1)

print('F1 scores for each fold:', scores)
print('Average F1 score:', scores.mean())

# 8-1. 최종 파이프라인 구성
SMOTE와 로지스틱 회귀를 파이프라인으로 구성하여 재학습합니다.

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('model', LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))